<a href="https://colab.research.google.com/github/hotbread213/createClass/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fin-ML/IVADO Finance and Insurance Workshop
## Reinforcement learning
Day 5 afternoon tutorial, March 8th 2019

1. A simple stock market problem
2. Q-learning
3. Q-learning with linear approximation
4. Exercice
5. Q-learning with linear approximation and experience replay

## 1. A simple stock market problem

There is a variety of finance problems that can be naturally formulated as control problems (finding an optimal policy for an MDP). Today we'll see an artificial example and look at different reinforcement learning algorithms to solve it.

We will first download the code for the environment.

In [0]:
!wget -O /content/environment.py https://www.dropbox.com/s/bfxh3nb4b1iiey7/environment.py

The concept of the game is as follows. You are a broker charged with selling 10 shares of a stock. The bid price follows a roughly sinusoidal pattern. At each timestep, you are given info about the state of the limit order book, and you have 20 timesteps to sell all your shares.

Notes:
- If at the end of the 20 timesteps you have shares left they will be sold at the current price. 
- If you try to sell more shares than you have left, your order will be corrected to your inventory.

Sounds good? Let's see this in action.


In [0]:
from environment import Environment

env = Environment()

Let's start a new episode. The environment returns the initial state $s_0$.

In [0]:
initial_state = env.new_episode()
return_ = 0
print(f"The initial state is\n\n    s_0 = {initial_state}")

The state $s_t$ at time $t$ is composed of the following features, on which we can base ourselves to take decisions.

*   __bid_price__: The bid price of the asset at time $t$.
*   __bid_volume__: The volume of bids at time $t$.
*  __spread__: The bid-ask spead at time $t$.
*   __ask_volume__: The volume of asks at time $t$.
*  __n_trades__: The number of trades between $t-1$ and $t$.
* __inventory__: The amount of shares that are left to be sold at time $t$.

In your opinion, what should we do now?
Let's assume for simplicity we can only sell 0-2 shares at a time.

In [0]:
ACTION = 2

state, reward, done = env.step(ACTION)
return_ += reward

print(f"We made a reward of {reward:.2f}$.")
if done:
    print(f"The episode is over.")
    print(f"We made a total return of {return_:.2f}$ over the episode.")
else:
    print(f"The total return so far is {return_:.2f}$.\n")
    print(f"The new state is\n\n     s_{env.timestep} = {state}.\n")
    print(f"What should we do now?")

Our goal is to teach an agent to maximize such returns. Reinforcement learning can help.

But first, it might be interesting to see what can of performance we can get from an agent that takes random decisions. What is the average return of such an agent?

In [0]:
import numpy as np

state = env.new_episode()
done = False
return_ = 0
while not done:
  action = int(np.random.choice(3))

  state, reward, done = env.step(action)
  return_ += reward
print(f"I made {return_:.2f}$.")

As you can see, the return is stochastic. Therefore, it would be more interesting to look at the distribution of returns of such an agent.

In [0]:
import matplotlib.pyplot as plt


returns = []
for _ in range(10000):
  state = env.new_episode()
  done = False
  return_ = 0
  while not done:
    action = int(np.random.choice(np.arange(3)))

    state, reward, done = env.step(action)
    return_ += reward
  returns.append(return_)

print(f"My average return was {np.mean(returns):.2f}$ with standard deviation ±{np.std(returns):.2f}$.")
plt.hist(returns)
plt.show()

Okay! Well, this can serve an a baseline - if we can't make more money than that on average, we are not very smart!

Let's see if reinforcement learning can do better.

## 2. Q-learning

As first RL algorithm, we will implement (tabular) Q-learning.
Tabular means there will be no statistical inference between states: every state will be treated distinctly, all equally dissimilar.

Our first obstacle is that the states are a mix of continuous and integer-valued features: we need to discretize them to apply tabular methods.


In [0]:
state_bins = [[0, 8, 20], 
              [0, 8, 20], 
              [0, 8, 20], 
              [0, 8, 20], 
              [0, 8, 20],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

def discretize(state):
  discretized_state = []
  for feature, bins in zip(state, state_bins):
    discretized_state.append(int(np.digitize(feature, bins))-1)
  return tuple(discretized_state)

print(f"Raw state:         {state}")
print(f"Discretized state: {discretize(state)}")

Here I used a simple binning system, where every feature is binary. This loses a lot of information but is a simple way to get started.

We are now ready to do Q-learning. We will attempt to estimate $Q_*(s, a)$, the expected return from taking action $a$ in state $s$ and hereafter behaving optimally. How can we compute such a function?

Well, we don't know much about $Q_*$, but we know it must satisfy **Bellman's equation**

$$Q_*(s, a) = \text{E}\bigg[R+\max_{a'}Q_*(S', a')\;\bigg\vert\;S=s, A=a\bigg]$$

with the conditions:
$$Q_*(s, \cdot) \equiv 0\qquad\text{for any terminal }s\in S.$$

This suggests an algorithm (Q-learning) as follows. First, we will create our Q-function table and initialize it randomly.

In [0]:
Q = np.random.rand(*(len(bins)-1 for bins in state_bins), 3)
print(f"Shape of Q: {Q.shape}")

Next, we will simulate runs of an agent that behaves as follows when in a state $s$:
- 90% of the time, it will pick the action $a^*=\arg\max_a Q(s, a)$;
- 10% of the time, it will pick a random action, $a\sim \text{Uniform}\{0,...,10\}$,

For example,

In [0]:
state = env.new_episode()
explore = np.random.rand() < 0.1
if explore:
  action = np.random.choice(np.arange(3))
else:
  action_values = Q[discretize(state) + (...,)]
  action = action_values.argmax()
print(f"In state   {state},\nI chose to sell {action} items")

We act accordingly...

In [0]:
next_state, reward, done = env.step(action)
print(f"I got {reward:.2f}$ in immediate reward,\nand the new state is {next_state}")

... and make a soft, Bellman-style update to our Q-function estimate,

$$Q(s, a) = 0.5\cdot Q(s,a) + 0.5\cdot\big[r+\max_{a'}Q(s', a')\big]$$

that is


In [0]:
Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                 + 0.5 * (reward + Q[discretize(next_state) + (...,)].max())

We can continue doing this until we reach the end of the episode,

In [0]:
while True:
  state = next_state
  explore = np.random.rand() < 0.1
  if explore:
    action = np.random.choice(np.arange(0, 3))
  else:
    action_values = Q[discretize(state) + (...,)]
    action = action_values.argmax()
  print(f"I chose to sell {action} items")

  next_state, reward, done = env.step(action)
  print(f"I got a reward of {reward:.2f}$")
  if done:
    print("End of episode")
    break

  Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                   + 0.5 * (reward + Q[discretize(next_state) + (...,)].max())

And now that we reached a terminal state, we should do a hard update

$$Q(s, a) = r$$

because we don't know much about the optimal Q, but we do know it should be zero for terminal states.

In [0]:
Q[discretize(state) + (action,)] = reward

In summary, this is what a training loop over a single episode looks like:

In [0]:
state = env.new_episode()
return_ = 0
while True:
  explore = np.random.rand() < 0.1
  if explore:
    action = np.random.choice(np.arange(3))
  else:
    action_values = Q[discretize(state) + (...,)]
    action = action_values.argmax()
  print(f"I chose to sell {action} items")

  next_state, reward, done = env.step(action)
  print(f"I got a reward of {reward:.2f}$")
  return_ += reward
  if not done:
    Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                     + 0.5 * (reward + Q[discretize(next_state) + (...,)].max())
    state = next_state
  else:
    Q[discretize(state) + (action,)] = reward
    print(f"End of episode, my return was {return_:.2f}")
    break

Progressively, the $Q$-function will magically start to converge towards the optimal $Q_*$.

(At least, optimal for a given discretization scheme.)

In [0]:
Q = np.random.rand(*(len(bins)-1 for bins in state_bins), 3)
for episode in range(5000):
  state = env.new_episode()
  return_ = 0
  while True:
    explore = np.random.rand() < 0.1
    if explore:
      action = np.random.choice(np.arange(3))
    else:
      action_values = Q[discretize(state) + (...,)]
      action = action_values.argmax()

    next_state, reward, done = env.step(action)
    return_ += reward
    if not done:
      Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                       + 0.5 * (reward + Q[discretize(next_state) + (...,)].max())
      state = next_state
    else:
      Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                       + 0.5 * reward
      break
  if episode % 1000 == 0:
    print(f"Episode {episode}, my return was {return_:.2f}")
print("Done!")

We can now check if our greedy agent (the one that always follows $a^*=\arg\max_a Q(s, a)$) does better than a random agent.

In [0]:
returns = []
for _ in range(10000):
  state = env.new_episode()
  done = False
  return_ = 0
  while not done:
    action_values = Q[discretize(state) + (...,)]
    action = action_values.argmax()

    state, reward, done = env.step(action)
    return_ += reward
  returns.append(return_)

print(f"My average return was {np.mean(returns):.2f}$ with standard deviation ±{np.std(returns):.2f}$.")
plt.hist(returns)
plt.show()

So it looks like we got some improvements - how about plotting return by episode? We could, say, train our Q-learning 10000 random agent like before, and every training episode we check the average return of our agent vs a random agent over 5 seeds.

In [0]:
def run_episode(agent):
  """ Runs an episode for an agent, returns the return."""
  state = env.new_episode()
  done = False
  return_ = 0
  while not done:
    if agent == "RL":
      action_values = Q[discretize(state) + (...,)]
      action = action_values.argmax()
    elif agent == "random":
      action = int(np.random.choice(np.arange(3)))

    state, reward, done = env.step(action)
    return_ += reward
  return return_


Q = np.random.rand(*(len(bins)-1 for bins in state_bins), 3)
RL_returns = []
random_returns = []
for episode in range(5000):
  # Training
  state = env.new_episode()
  while True:
    explore = np.random.rand() < 0.1
    if explore:
      action = np.random.choice(np.arange(3))
    else:
      action_values = Q[discretize(state) + (...,)]
      action = action_values.argmax()

    next_state, reward, done = env.step(action)
    if not done:
      Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                       + 0.5 * (reward + Q[discretize(next_state) + (...,)].max())
      state = next_state
    else:
      Q[discretize(state) + (action,)] = 0.5 * Q[discretize(state) + (action,)] \
                                       + 0.5 * reward
      break
  
  # Evaluation
  RL_return = np.mean([run_episode("RL") for _ in range(5)])
  RL_returns.append(RL_return)
  random_return = np.mean([run_episode("random") for _ in range(5)])
  random_returns.append(random_return)
  if episode % 1000 == 0:
    print(f"Episode {episode}, RL agent return was {RL_return:.2f}"
          f", random agent return was {random_return:.2f}")
print("Done!")

If we plot the returns this is what we find!

In [0]:
smoothed_RL_returns = np.convolve(RL_returns, np.ones((100,))/100, mode='valid')
smoothed_random_returns = np.convolve(random_returns, np.ones((100,))/100, mode='valid')

plt.plot(np.arange(len(smoothed_RL_returns)), smoothed_RL_returns, '-', 
         np.arange(len(smoothed_random_returns)), smoothed_random_returns, '-')
plt.show()

## 3. Q-learning with linear approximation

To use Q-learning we had to discretize our state space. This is problematic because you lose information: in the worst case, if you get the bins completely wrong, all the states could be sent to the same bin and you might destroy all the information! Moreover, this adds additional hyperparameters and reinforcement learning algorithms, in general, tend to be very sentitive to hyperparameters.

To address this, we will make use of function approximation and see what kind of improvements it might offer. In this case it will be a linear model, the easiest kind.

Q-learning with linear function approximation works as follows. Whereas $Q(s,a)$ used to be a table, now for every $a$ $s\mapsto Q(s, a)$ will be a linear regression model. We will explore the environment the usual way,
- 90% of the time, pick the action $a^*=\arg\max_a Q(s, a)$,
- 10% of the time, pick a random action, $a\sim \text{Uniform}\{0,...,10\}$,
collection transitions $(s,a,r,s')$ along the way.

What differs is how we update the Q-function after observing the transition. Usually, linear models are fitted offline, with all the data in one shot, but here we will keep observing new data and we do not want to forget older data. What is typical then is to train the linear model using an online scheme, such as stochastic gradient descent. Namely, after observing the transition $(s,a,r,s')$, we will update the linear model $Q(\cdot, a)$ with a gradient descent step on a single $(x,y)$ pair

$$\big(s, r+\max_{a'}Q(s',a')\big).$$

Let's see this in practice. Let's first create our linear models.

In [0]:
from sklearn.linear_model import SGDRegressor

Q = []
for _ in range(3):
  Q_a = SGDRegressor(alpha=0.05)
  Q_a.partial_fit(np.zeros((1, 6)), [100])
  Q.append(Q_a)

I decided to initialize them by a single gradient descent step on the pair $(0, 100)$: that is, the zero state will be initialized to 100$.

We can now start the usual way - create an environment and take an action, say, sell one share.

In [0]:
state = env.new_episode()
action = 2
next_state, reward, done = env.step(action)

After observing the transition, we can now update the relevant linear model.

In [0]:
next_values = np.array([Q[action].predict(next_state.reshape(1, -1)) 
                        for action in range(3)])
Q[action].partial_fit(state.reshape(1, -1), [reward + next_values.max()])

And we can then continue traning over the episode, remembering that terminal states have zero value by construction.

In [0]:
state = env.new_episode()
return_ = 0
while True:
  explore = np.random.rand() < 0.1
  if explore:
    action = np.random.choice(np.arange(3))
  else:
    action_values = np.array([Q[action].predict(state.reshape(1, -1)) 
                              for action in range(3)])
    action = action_values.argmax()
  print(f"I chose to sell {action} items")

  next_state, reward, done = env.step(action)
  print(f"I got a reward of {reward:.2f}$")
  return_ += reward
  if not done:
    next_values = np.array([Q[action].predict(next_state.reshape(1, -1)) 
                      for action in range(3)])
    Q[action].partial_fit(state.reshape(1, -1), [reward + next_values.max()])
    state = next_state
  else:
    Q[action].partial_fit(state.reshape(1, -1), [reward])
    print(f"End of episode, my return was {return_:.2f}")
    break

Pretty good! And now, we can repeat this training over many episodes to train our agent.
This is what we obtain.

In [0]:
Q = []
for _ in range(3):
  Q_a = SGDRegressor(alpha=0.05)
  Q_a.partial_fit(np.zeros((1, 6)), [100])
  Q.append(Q_a)


def run_episode(agent):
  """ Runs an episode for an agent, returns the return."""
  state = env.new_episode()
  done = False
  return_ = 0
  while not done:
    if agent == "RL":
      action_values = np.array([Q[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()
    elif agent == "random":
      action = int(np.random.choice(np.arange(3)))

    state, reward, done = env.step(action)
    return_ += reward
  return return_


RL_returns = []
random_returns = []
for episode in range(1000):
  # Training
  state = env.new_episode()
  while True:
    explore = np.random.rand() < 0.1
    if explore:
      action = np.random.choice(np.arange(3))
    else:
      action_values = np.array([Q[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()

    next_state, reward, done = env.step(action)
    if not done:
      next_values = np.array([Q[action].predict(next_state.reshape(1, -1)) 
                        for action in range(3)])
      Q[action].partial_fit(state.reshape(1, -1), [reward + next_values.max()])
      state = next_state
    else:
      Q[action].partial_fit(state.reshape(1, -1), [reward])
      break
  
  # Evaluation
  RL_return = np.mean([run_episode("RL") for _ in range(5)])
  RL_returns.append(RL_return)
  random_return = np.mean([run_episode("random") for _ in range(5)])
  random_returns.append(random_return)
  if episode % 200 == 0:
    print(f"Episode {episode}, RL agent return was {RL_return:.2f}"
          f", random agent return was {random_return:.2f}")
print("Done!")

And now we can plot the training curve...

In [0]:
smoothed_RL_returns = np.convolve(RL_returns, np.ones((100,))/100, mode='valid')
smoothed_random_returns = np.convolve(random_returns, np.ones((100,))/100, mode='valid')

plt.plot(np.arange(len(smoothed_RL_returns)), smoothed_RL_returns, '-', 
         np.arange(len(smoothed_random_returns)), smoothed_random_returns, '-')
plt.show()

... and the histogram of returns.

In [0]:
returns = [run_episode("RL") for _ in range(10000)]

print(f"My average return was {np.mean(returns):.2f}$ with standard deviation ±{np.std(returns):.2f}$.")
plt.hist(returns)
plt.show()

What can we conclude? Well, first, it didn't really change the performance of the optimal policy. But, interestingly, the training was _much_ faster.

## 4. Exercice

We will now try to turn this into practice. I made an exercice environment you can play with, which you can load like this:

In [0]:
from environment import ExerciceEnvironment
  
env = ExerciceEnvironment()

The environment is very similar to the one you just used before, but with small modifications:
- There are now 7 state features;
- The actions are now -1, 0 and 1.


In [0]:
state = env.new_episode()
print(f"State: {state}\n")

action = -1
next_state, reward, done = env.step(action)
print(f"Next state: {next_state}\nReward: {reward:.2f}$\nDone: {done}")

Now try to code a Q-learning algorithm. 

How good is your resulting agent? How fast did it learn?
How far can you push it? Here are some hyperparameters to play with:
- The SGD learning rate (default $\alpha=0.05$).
- The initial Q-function value (default 100 dollars)
- The exploration rate (default $\epsilon=0.1$)

Good luck!


## 5. Q-learning with linear approximation and experience replay

Finally, we will cover experience replay, a trick that accelerates and stabilitizes Q-learning.
A disadvantage of Q-learning, as shown before, is that samples are used once and then thrown away. This is somewhat wasteful.

Experience replay is a simple trick to increase sample efficiency: we store transitions in a buffer, and when updating we batch some of these samples for an update.

Let's see this in practice. First, let's create our environment again.

In [0]:
env = Environment()

Q = []
for _ in range(3):
  Q_a = SGDRegressor(alpha=0.05)
  Q_a.partial_fit(np.zeros((1, 6)), [100])
  Q.append(Q_a)

Let's now take an action, and learn from the transition like before, but! We also store it in a memory buffer.

In [0]:
memory = []

state = env.new_episode()
action = int(np.random.choice(np.arange(3)))
next_state, reward, done = env.step(action)
memory.append((state, action, reward, next_state, float(done)))  # <------------

next_values = np.array([Q[action].predict(next_state.reshape(1, -1)) 
                  for action in range(3)])
Q[action].partial_fit(state.reshape(1, -1), [reward + next_values.max()])
state = next_state

Then at the next transition, we not only learn from the new transition but also the old one.

In [0]:
action = int(np.random.choice(np.arange(3)))
next_state, reward, done = env.step(action)
memory.append((state, action, reward, next_state, float(done)))

states, actions, rewards, next_states, dones = map(np.stack, zip(*memory))
next_values = np.array([Q[action].predict(next_states) 
                  for action in range(3)])
for action in actions:
  Q[action].partial_fit(states[actions == action], 
                        (rewards + (1-dones) * next_values.max(axis=0))[actions == action])
state = next_state

Eventually the memory buffer will grow quite big, making updates slow and reducing the weight given to new observations. To counter-balance that, it is typical to:
- Flush old transitions.
- Sample from the buffer.

Putting everything together in a loop, this is what we obtain.

In [0]:
QER = []
for _ in range(3):
  Q_a = SGDRegressor(alpha=0.05)
  Q_a.partial_fit(np.zeros((1, 6)), [100])
  QER.append(Q_a)


Q = []
for _ in range(3):
  Q_a = SGDRegressor(alpha=0.05)
  Q_a.partial_fit(np.zeros((1, 6)), [100])
  Q.append(Q_a)

 
 

def run_episode(agent):
  """ Runs an episode for an agent, returns the return."""
  state = env.new_episode()
  done = False
  return_ = 0
  while not done:
    if agent == "RLER":
      action_values = np.array([QER[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()
    if agent == "RL":
      action_values = np.array([Q[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()
    elif agent == "random":
      action = int(np.random.choice(np.arange(3)))

    state, reward, done = env.step(action)
    return_ += reward
  return return_


RLER_returns = []
RL_returns = []
random_returns = []
memory = []
for episode in range(1000):
  # Experience replay training
  state = env.new_episode()
  while True:
    explore = np.random.rand() < 0.1
    if explore:
      action = np.random.choice(np.arange(3))
    else:
      action_values = np.array([QER[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()

    next_state, reward, done = env.step(action)
    memory.append((state, action, reward, next_state, float(done)))
    memory = memory[-128:]
    sample = [memory[index] for index in np.random.choice(len(memory),min(len(memory), 32), replace=False)]
    states, actions, rewards, next_states, dones = map(np.stack, zip(*sample))
    next_values = np.array([QER[action].predict(next_states) 
                      for action in range(3)])
    for action in actions:
      QER[action].partial_fit(states[actions == action], 
                            (rewards + (1-dones) * next_values.max(axis=0))[actions == action])
    if not done:
      state = next_state
    else:
      break
   
  
  # Regular training
  state = env.new_episode()
  while True:
    explore = np.random.rand() < 0.1
    if explore:
      action = np.random.choice(np.arange(3))
    else:
      action_values = np.array([Q[action].predict(state.reshape(1, -1)) 
                                for action in range(3)])
      action = action_values.argmax()

    next_state, reward, done = env.step(action)
    if not done:
      next_values = np.array([Q[action].predict(next_state.reshape(1, -1)) 
                        for action in range(3)])
      Q[action].partial_fit(state.reshape(1, -1), [reward + next_values.max()])
      state = next_state
    else:
      Q[action].partial_fit(state.reshape(1, -1), [reward])
      break
  
  
  
  # Evaluation
  RLER_return = np.mean([run_episode("RLER") for _ in range(5)])
  RLER_returns.append(RLER_return)
  RL_return = np.mean([run_episode("RL") for _ in range(5)])
  RL_returns.append(RL_return)
  random_return = np.mean([run_episode("random") for _ in range(5)])
  random_returns.append(random_return)
  if episode % 200 == 0:
    print(f"Episode {episode}"
          f", RL agent with ER return was {RLER_return:.2f}"
          f", RL agent return was {RL_return:.2f}"
          f", random agent return was {random_return:.2f}")
print("Done!")

And the training curves look like this:

In [0]:
smoothed_RLER_returns = np.convolve(RLER_returns, np.ones((100,))/100, mode='valid')
smoothed_RL_returns = np.convolve(RL_returns, np.ones((100,))/100, mode='valid')
smoothed_random_returns = np.convolve(random_returns, np.ones((100,))/100, mode='valid')

plt.plot(np.arange(len(smoothed_RLER_returns)), smoothed_RLER_returns, '-', 
         np.arange(len(smoothed_RL_returns)), smoothed_RL_returns, '-', 
         np.arange(len(smoothed_random_returns)), smoothed_random_returns, '-')
plt.show()

So much more efficient!